# Esercizio 5

Prendere in input un file in formato `GTF` (Gene Transfer Format) e il file `FASTA` della genomica di riferimento.

In dipendenza di una determinata scelta, ricostruire tutte le sequenze di un certo tipo (trascritto o coding sequence o 5'UTR o 3'UTR) che sono annotate nel file GTF e salvarle, in formato FASTA, in un file.

L'*header* `FASTA` di ogni sequenza prodotta deve contenere:

- il nome del gene di riferimento
- l’identificatore del trascritto di riferimento
- la lunghezza della sequenza
- il tipo di sequenza (trascritto oppure CDS oppure 5'UTR oppure 3'UTR)
- lo *strand*
    
Esempio di *header* per un trascritto:
         
    >ARHGAP4; U52112.4-003; len=3235 type=exon; strand=-

Esempio di *header* per una CDS:

    >AVPR2; U52112.2-003; len=642; type=CDS; strand=+
    
Se si sceglie di ricostruire le *coding sequences*, produrre in standard output la loro separazione in codoni.
   
***


## Dataset in input

In [ ]:
gtf_file_name = './input.gtf'
reference_file_name = './ENm006.fa'

## Tipo di sequenza da ricostruire

Valori possibili:

- `exon`, se si vogliono ricostruire i trascritti
- `CDS`, se si vogliono ricostruire le coding sequences
- `5UTR`, se si vogliono ricostruire le 5'UTR
- `3UTR`, se si vogliono ricostruire le 3'UTR

In [ ]:
selected_feature = 'CDS'

## Importare il modulo `re`

In [ ]:
import re

## Definire la funzione per estrarre il gene e il trascritto associati a una certa *feature*

La funzione deve prendere come argomento un *record* GTF e restituire la tupla `(gene_name, transcript_id)` contenente il nome del gene e l'identificatore del trascritto associati al *record*.

**NOTA BENE**: gli attributi all'interno del nono campo di un file `GTF` non hanno ordine fisso all'interno del campo.

In [ ]:
def get_gene_and_transcript(gtf_record):
    gene_name = re.search(r'gene_id\s"(.+?)";', gtf_record).group(1)
    transcript_id = re.search(r'transcript_id\s"(.+?)";', gtf_record).group(1)
    
    return (gene_name, transcript_id)

## Definire la funzione che formatta una sequenza in `FASTA` 

Fare riferimento all'esercizio 3.

In [ ]:
def format_fasta(header, sequence, record = 80):    
    p = re.compile('\w{,' + str(record) + '}')
    return header + '\n' + '\n'.join(re.findall(p, sequence))

## Definire la funzione per restituire il *reverse and complement* di una sequenza nucleotidica

La funzione deve prendere come argomento una sequenza nucleotidica e restituire il *reverse and complement* della sequenza in lettere maiuscole.

In [ ]:
def reverse_complement(nucleotide_sequence):
    complement_dict = {'A' : 'T', 'T' : 'A', 'C' : 'G', 'G' : 'C'}
    
    return ''.join(complement_dict[b] for b in nucleotide_sequence[::-1].upper())

In [ ]:
reverse_complement('AAACCC')

## Definire la funzione che ricostruisce la sequenza di un certo tipo

La funzione deve prendere i seguenti argomenti:
- la lista delle *features* che compongono la sequenza da ricostruire
- la sequenza della genomica di riferimento
- lo *strand*

Prevedere che ogni *feature* nella lista sia una tupla `(start, end, frame)` di dimensione tre, in cui i primi due valori sono le posizioni 1-based di inizio e fine della *feature* sulla genomica di riferimento. Il terzo elemento `frame` è diverso da `.` (dot) solo se si sta ricostruendo una coding sequence.

In [ ]:
def reconstruct_sequence(feature_list, reference_sequence, strand):
    sequence = ''
    feature_list.sort()
    for x in feature_list:
        sequence = sequence + reference_sequence[x[0]-1: x[1]]
        
    if strand == '-':
        return reverse_complement(sequence)
    else:
        return sequence

## Recuperare la genomica di riferimento

In [ ]:
with open(reference_file_name, 'r') as input_file:
    reference_sequence = ''.join(input_file.readlines()[1:]).replace('\n', '')

In [ ]:
reference_sequence

## Ottenere dal file GTF i *record* che servono per ricostruire le sequenze del tipo scelto

In [ ]:
with open(gtf_file_name, 'r') as input_file:
    selected_gtf_records = [record for record in input_file.readlines() if record.split('\t')[2] == selected_feature]

In [ ]:
selected_gtf_records

## Costruire il dizionario degli *strand*

Solo per i geni per cui è stata annotata una sequenza del tipo scelto, creare il dizionario:

- **chiave**: nome del gene
- **valore**: *strand* del gene (rispetto al *reference*)

In [ ]:
strand_dict = {}

for record in selected_gtf_records:
    strand = record.split('\t')[6]
    
    (gene_name, transcript_id) = get_gene_and_transcript(record)
    
    strand_dict[gene_name] = strand

In [ ]:
strand_dict

## Ricostruire le sequenze del tipo scelto

a) Costruire il dizionario degli identificatori dei trascritti:

   - **chiave**: nome del gene
   - **valore**: insieme degli identificatori dei trascritti per cui è annotata la sequenza del tipo scelto

In [ ]:
gene_dict = {}

for record in selected_gtf_records:
    (gene_name, transcript_id) = get_gene_and_transcript(record)
    
    value = gene_dict.get(gene_name, set())
    value.add(transcript_id)
    gene_dict[gene_name] = value

In [ ]:
gene_dict

b) Costruire il dizionario delle *features* per trascritto:

   - **chiave**: identificatore del trascritto
   - **valore**: lista delle *features*, associate al trascritto, che compongono la sequenza del tipo scelto
   
Ogni *feature* nella lista deve essere rappresentata dalla tupla `(start, end, frame)` contenente le posizioni 1-based di inizio e fine della *feature* sul *reference* e il valore di *frame* della *feature* (che sarà un *dot* - assenza di informazione - nel caso si scelga di ricostruire sequenze diverse dalle *coding sequences*.

In [ ]:
composition_dict = {}

for record in selected_gtf_records:
    (gene_name, transcript_id) = get_gene_and_transcript(record)
    
    value = composition_dict.get(transcript_id, [])
    
    fields = record.split('\t')
    
    tupla = (int(fields[3]), int(fields[4]), int(fields[7]))
    value.append(tupla)
    
    composition_dict[transcript_id] = value

In [ ]:
composition_dict

c) Se si stanno ricostruendo coding sequences, ottenere (dal precedente) il dizionario dei valori di *frame* della prima *feature* CDS:

   - **chiave**: identificatore di trascritto
   - **valore**: *frame* della prima *feature* CDS

In [ ]:
frame_dict = {}

if selected_feature == 'CDS':
    
    for gene_name in gene_dict:
        for transcript_id in gene_dict[gene_name]:
            feature_list = composition_dict[transcript_id]
            sorted_list = sorted(feature_list)
                
            if strand_dict[gene_name] == '+':
                frame = sorted_list[0][2]
            else:
                frame = sorted_list[-1][2]
                    
            frame_dict[transcript_id] = int(frame)

In [ ]:
frame_dict

d) Ricostruire le sequenze e salvarle in un file `FASTA`

Esempio di *header*:

    >ARHGAP4; U52112.4-003; len=3235; type=[exon|CDS]; strand=-
    
**NB**: se si stanno ricostruendo le *coding sequences*, stampare anche in standard output la separazione in codoni tenendo conto del *frame* della prima *feature*.

In [ ]:
with open('./output_file.fa', 'w') as output_file:
    for gene_name in gene_dict:
        for transcript_id in gene_dict[gene_name]:
            r_sequence = reconstruct_sequence(composition_dict[transcript_id], reference_sequence, strand_dict[gene_name])
            
            header = '>' + gene_name + '-' + transcript_id
            
            output_file.write(format_fasta(header, r_sequence, record = 60))
            
            if selected_feature == 'CDS':
                frame = frame_dict[transcript_id]
                
                print(header)
                
                if frame != 0:
                    print(r_sequence[:frame], end = ' ')
                    
                print(' '.join(re.findall(r'.{,3}', r_sequence[frame:])))